In [2]:
!pip install -q torch


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import ParameterGrid
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder




In [42]:
# Load the dataset
data = pd.read_csv('synthetic_dataSet.csv')

# Separate features and target variable
X = data.drop(columns=['commission'])
y = data['commission']

# Assuming X_train is already defined
scaler = StandardScaler()

# Encode categorical variables
categorical_features = ['policy_type', 'region', 'add_ons']
encoder = LabelEncoder()
for feature in categorical_features:
    X[feature] = encoder.fit_transform(X[feature])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

# Print the shape of X_train to verify it
print("Shape of X_train_scaled:", X_train_scaled.shape)
print("Shape of X_test_scaled:", X_test_scaled.shape)
print("Shape of X_train_tensor:", X_train_tensor.shape)
print("Shape of X_test_tensor:", X_test_tensor.shape)


Shape of X_train_scaled: (8000, 4)
Shape of X_test_scaled: (2000, 4)
Shape of X_train_tensor: torch.Size([8000, 4])
Shape of X_test_tensor: torch.Size([2000, 4])


In [35]:
# Define the neural network model with L2 regularization
class CommissionPredictor(nn.Module):
    def __init__(self, input_dim, num_hidden_layers, hidden_layer_size, activation_fn, l2_reg):
        super(CommissionPredictor, self).__init__()
        self.num_hidden_layers = num_hidden_layers
        self.hidden_layer_size = hidden_layer_size
        self.activation_fn = activation_fn  # Store activation function
        self.l2_reg = l2_reg

        # Define the layers
        self.input_layer = nn.Linear(input_dim, hidden_layer_size)
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_layer_size, hidden_layer_size) for _ in range(num_hidden_layers)])
        self.output_layer = nn.Linear(hidden_layer_size, 1)

    def forward(self, x):
        # Forward pass through input layer
        x = self.input_layer(x)
        x = self.activation_fn(x)  # Apply activation function to input layer output

        # Forward pass through hidden layers
        for _ in range(self.num_hidden_layers):
            x = self.activation_fn(self.hidden_layers[_](x))  # Apply activation function to hidden layer output

        # Forward pass through output layer
        x = self.output_layer(x)
        return x

    def l2_loss(self):
        l2_loss = 0.0
        for param in self.parameters():
            l2_loss += torch.sum(param ** 2)
        return l2_loss * self.l2_reg


In [36]:
# Define hyperparameters to search
param_grid = {
    'num_hidden_layers': [1, 2, 3],           # Number of hidden layers
    'hidden_layer_size': [64, 128, 256],      # Size of hidden layers
    'activation_fn': [nn.ReLU(), nn.LeakyReLU()], # Activation functions
    'lr': [0.001, 0.01],                       # Learning rate
    'num_epochs': [50, 100],                   # Number of epochs
    'batch_size': [32, 64],                     # Batch size
    'l2_reg': [0.001, 0.01]                    # L2 regularization strength
}


In [43]:
# Convert hyperparameter grid into a list of dictionaries
param_list = list(ParameterGrid(param_grid))

# Assuming X_train and y_train are defined
# Define dataset dimensions
input_dim = X_train.shape[1]  # Number of input features

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

print("Shape of X_train_tensor:", X_train_tensor.shape)
print("Shape of y_train_tensor:", y_train_tensor.shape)
print("Shape of X_test_tensor:", X_test_tensor.shape)
print("Shape of y_test_tensor:", y_test_tensor.shape)

Shape of X_train_tensor: torch.Size([8000, 4])
Shape of y_train_tensor: torch.Size([8000])
Shape of X_test_tensor: torch.Size([2000, 4])
Shape of y_test_tensor: torch.Size([2000])


In [44]:
# Train and evaluate model for each set of hyperparameters
best_val_loss = np.inf
best_model = None
for params in param_list:
    # Initialize model, loss function, and optimizer
    model = CommissionPredictor(input_dim, params['num_hidden_layers'], params['hidden_layer_size'], params['activation_fn'], params['l2_reg'])
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['lr'])

    # Training loop
    for epoch in range(params['num_epochs']):
        for i in range(0, len(X_train_tensor), params['batch_size']):
            optimizer.zero_grad()
            batch_X = X_train_tensor[i:i+params['batch_size']]
            batch_y = y_train_tensor[i:i+params['batch_size']]
            outputs = model(batch_X)
            loss = criterion(outputs.squeeze(), batch_y) + model.l2_loss()  # Add L2 regularization to the loss
            loss.backward()
            optimizer.step()

    # Evaluate the model on the validation set
    with torch.no_grad():
        predictions = model(X_test_tensor)
        val_loss = criterion(predictions.squeeze(), y_test_tensor)

    # Update best model if validation loss is lower
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

In [216]:
# Assuming X_test and y_test are defined
# Convert data to PyTorch tensors
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Evaluate the best model on the test set
with torch.no_grad():
    predictions = best_model(X_test_tensor)
    test_loss = criterion(predictions.squeeze(), y_test_tensor)
    print(f'Best Validation Loss: {best_val_loss.item():.4f}')
    print(f'Test Loss: {test_loss.item():.4f}')


Epoch [1/50], Loss: 67862.7188
Epoch [2/50], Loss: 19761.1660
Epoch [3/50], Loss: 10239.9111
Epoch [4/50], Loss: 9601.0879
Epoch [5/50], Loss: 9521.9746
Epoch [6/50], Loss: 9498.9326
Epoch [7/50], Loss: 9623.2959
Epoch [8/50], Loss: 9522.7764
Epoch [9/50], Loss: 9396.9863
Epoch [10/50], Loss: 9406.6484
Epoch [11/50], Loss: 9382.9844
Epoch [12/50], Loss: 9376.6104
Epoch [13/50], Loss: 9365.5342
Epoch [14/50], Loss: 9359.1807
Epoch [15/50], Loss: 9355.2959
Epoch [16/50], Loss: 9344.7002
Epoch [17/50], Loss: 9330.9434
Epoch [18/50], Loss: 9321.7236
Epoch [19/50], Loss: 9326.3623
Epoch [20/50], Loss: 9374.5586
Epoch [21/50], Loss: 9447.8086
Epoch [22/50], Loss: 9491.4893
Epoch [23/50], Loss: 9460.9824
Epoch [24/50], Loss: 9398.8877
Epoch [25/50], Loss: 9363.7354
Epoch [26/50], Loss: 9355.1357
Epoch [27/50], Loss: 9352.1201
Epoch [28/50], Loss: 9348.6143
Epoch [29/50], Loss: 9347.0518
Epoch [30/50], Loss: 9348.3271
Epoch [31/50], Loss: 9348.6162
Epoch [32/50], Loss: 9353.7773
Epoch [33/50],

In [193]:
# Save the best model
model_filename = "commission_predictor_model.pth"
torch.save(best_model.state_dict(), model_filename)

Test Loss: 10953.5869
